<a href="https://colab.research.google.com/github/AnnSenina/Python_FLAS_2025/blob/main/notebooks/%D0%9F%D0%B0%D1%80%D1%81%D0%B8%D0%BD%D0%B3_%D0%BF%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D0%BA%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Парсинг_ практика

In [ ]:
# импортируем модули в тетрадку

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm import tqdm

Гайды и туториалы

- [документация requests и быстрый гайд](https://requests.readthedocs.io/en/master/user/quickstart/)


- [документация Beautiful Soup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)

- [text-only](https://sjmulder.nl/en/textonly.html) страница содержит веб-сайты, чтобы легко начать парсить

- [здесь](https://www.york.ac.uk/teaching/cws/wws/webpage1.html) можно почитать про структуру html подробнее


- [здесь](https://www.w3schools.com/html/html_examples.asp) еще и потренироваться в режиме онлайн (с этой ссылки мы начали занятие)

### Задания

1. Спарсите таблички со страницы https://en.wikipedia.org/wiki/List_of_nuclear_weapons_tests:

- Worldwide nuclear testing totals by country
- Worldwide nuclear test with a yield of 1.4 Mt TNT equivalent and more
- Largest fission bomb tests

Каждую табличку можно передать в датафрейм и сохранить в отдельный файл (удобнее спарсить ВСЕ с помощью pandas, потом выбрать 3 нужных вручную))

In [ ]:
# ваш код


2. Парсинг цитат с сайта:

https://quotes.toscrape.com

- Сгенерируйте ссылки на все 10 страниц цитат и сохраните все цитаты в текстовый файл

- Затем попробуйте сохранить в датафрейм следующую информацию: автор, текст цитаты, тэги

- соберите всю информацию об авторах: имя, когда человек родился, его биография, ссылка на его страницу

In [ ]:
# ваш код


3. Парсинг информации в датафрейм

- соберите ссылки на все 50 страниц с сайта [Books to Scrape](https://books.toscrape.com/)
- с каждой страницы всоберите все ссылки на отдельные книги (на этом шаге - **только** ссылки!)
- затем пройдите по собранным ссылкам на отдельные книги соберите таблицу со следующими столбцами: ссылка, название книги, цена, текстовое описание

**Важно!** текстовое описание должно вызвать проблему - можно сначала собрать без него, потом отловить через try / except и придумать, как обойти

In [ ]:
# ваш код
# шаг 1 - ссылки на все страницы с книгами



### Все решения

Задание 1

In [ ]:
# @title
res = pd.read_html('https://en.wikipedia.org/wiki/List_of_nuclear_weapons_tests')
print(len(res))

In [ ]:
# @title

df1 = pd.DataFrame(res[1])
df2 = pd.DataFrame(res[2])
df3 = pd.DataFrame(res[3])
# табличек много, придется проверить первые несколько для поиска нужных

Задание 2


In [ ]:
# @title
# нагенерируем ссылок:
all_links = []
for i in range(1, 11):
  all_links.append(f'https://quotes.toscrape.com/page/{i}')

all_quotes = []
for i in all_links:
  page = requests.get(i)
  soup = BeautifulSoup(page.text)
  for qu in soup.find_all('span', {'class' : 'text', 'itemprop' : 'text'}):
    all_quotes.append(qu.text.strip())
print(*all_quotes[:5], sep='\n')
print('всего цитат:', len(all_quotes))

with open('quotes.txt', 'w', encoding='utf-8') as f:
  for i in all_quotes:
    print(i, file = f)

In [ ]:
# @title
texts = []
authors = []
tags = []
for i in all_links:
  page = requests.get(i)
  soup = BeautifulSoup(page.text)
  for qu in soup.find_all('span', {'class' : 'text', 'itemprop' : 'text'}):
    texts.append(qu.text.strip())
  for au in soup.find_all('small', {'class' : 'author', 'itemprop' : 'author'}):
    authors.append(au.text.strip())
  for t in soup.find_all('div', {'class' : 'tags'}):
    tags.append(t.text.replace('Tags:', '').replace('\n', ' ').strip())
quotes = pd.DataFrame([texts, authors, tags])
quotes = quotes.T
quotes.columns = ['quote', 'author', 'tags']
quotes

In [ ]:
# @title
authors_links = []
for i in all_links:
  page = requests.get(i)
  soup = BeautifulSoup(page.text)
  for link in soup.find_all('a'):
    if 'author' in link.get('href') and 'https://quotes.toscrape.com' + link.get('href') not in authors_links:
       authors_links.append('https://quotes.toscrape.com' + link.get('href'))
len(authors_links) # авторы часто повторяются!

In [ ]:
# @title
all_data = []
for i in authors_links:
  try:
    page = requests.get(i)
    soup = BeautifulSoup(page.text)
    name = soup.find('h3', {'class':"author-title"}).text.strip()
    born = soup.find('span', {'class':"author-born-date"}).text.strip()
    place = soup.find('span', {'class':"author-born-location"}).text.strip()
    bio = soup.find('div', {'class':"author-description"}).text.strip()
    author = [name, born, place, bio, i]
    all_data.append(author)
  except:
    print(i, 'не сработало') # попала лишняя ссылка, но не мешает
authors_df = pd.DataFrame(all_data)
authors_df.columns = ['name', 'born', 'place', 'bio', 'link']
authors_df

Задание 3

In [ ]:
# @title
# нагенерируем ссылок:
all_links = []
for i in range(1, 51):
  all_links.append(f'https://books.toscrape.com/catalogue/page-{i}.html')

book_links = []
for i in all_links:
  page = requests.get(i)
  soup = BeautifulSoup(page.text)
  for link in soup.find_all('a'):
    if link.get('title') != None:
      book_links.append('https://books.toscrape.com/catalogue/' + link.get('href'))
print(len(book_links))

In [ ]:
# @title
books_data = []
for i in tqdm(range(len(book_links))):
  try:
    page = requests.get(book_links[i])
    soup = BeautifulSoup(page.text)
    title = soup.find('h1').text.strip()
    price = soup.find('p', {'class':"price_color"}).text.strip()
    desc = soup.find('p', {'class':None}).text.strip() # оказалось, что не у всех есть описание!
    books_data.append([title, price, desc, book_links[i]])
  except:
    print(book_links[i])

In [ ]:
# @title
books_data = []
for i in tqdm(range(len(book_links))):
  try:
    page = requests.get(book_links[i])
    soup = BeautifulSoup(page.text)
    title = soup.find('h1').text.strip()
    price = soup.find('p', {'class':"price_color"}).text.strip()
    if soup.find('p', {'class':None}) != None: # исправляем
      desc = soup.find('p', {'class':None}).text.strip()
    else:
      desc = None
    books_data.append([title, price, desc, book_links[i]])
  except:
    print(book_links[i])

In [ ]:
# @title
books_df = pd.DataFrame(books_data)
books_df.columns = ['title', 'price', 'description', 'link']
books_df